# Creating Mean Predictor Maps

This Jupyter Notebook gives you the possiblity to calculate mean predictor maps for true positive and false negative predictions.

Model Runs and stations need to be specified. 


In [1]:
#---
# Modules
#---
import numpy as np
import xarray as xr
import pandas as pd
import pickle
import matplotlib.pyplot as plt
import time

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_curve, roc_auc_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler


from data import data_loader
from data import gesla_preprocessing
from data import era5_preprocessing
from data import preprocessing
from data import saver
from data import visualisation

from models import modelfit
from models import evaluation
from models import loader

In [ ]:
def load_GESLA(station_names, season, detrend_type, percentile):
    gesla_predictand = data_loader.load_gesla(station_names)
    station_positions = gesla_preprocessing.station_position(gesla_predictand, station_names)
    gesla_predictand = gesla_preprocessing.select_season(gesla_predictand, season)
    gesla_predictand = gesla_preprocessing.get_analysis(gesla_predictand)
    gesla_predictand = gesla_predictand["sea_level"] # Select values
    gesla_predictand = gesla_preprocessing.detrend_signal(gesla_predictand, type_=detrend_type) 
    gesla_predictand = gesla_preprocessing.apply_dummies(gesla_predictand, percentile=percentile, level="station")
    gesla_predictand = gesla_predictand.to_xarray()
    
    return gesla_predictand, station_positions

def load_ERA5(range_of_years, subregion, season, predictor, preprocess="preprocess1"):
    era5_predictor = data_loader.load_daymean_era5(range_of_years, subregion, season, predictor, preprocess) 
    era5_predictor = preprocessing.convert_timestamp(era5_predictor, dim="time")

    # Convert predictor sp.unit from Pa to hPA
    #---
    if predictor == "sp":
        with xr.set_options(keep_attrs=True):
            old_unit = era5_predictor.attrs["units"]
            era5_predictor = era5_predictor / 100
            era5_predictor.attrs["units"] = "hPa"
            new_unit = era5_predictor.attrs["units"]
            print(f"Converted units of {predictor} from {old_unit} to {new_unit}")

    return era5_predictor

def convert_data(X, Y,):

            # Convert to format needed for model fit
            #--- 
            n_pfs = 0

            X = np.array(X)
            Y = np.array(Y) 
            Y = Y[0, :] # Assume all timeseries are the same for the predictors.

            # Reshape for model input
            #---
            print(f"Reshape for model input")

            ndim = Y.shape[0]

            X = X.swapaxes(0, 1) # Put time dimension to front

            print(X.shape) # (time, timelags, predictor_combination, lon?, lat?)

            X = X.reshape(ndim, -1) # Reshapes into (time, pred1_lonlats:pred2_lonlats:...:predn_lonlats)
            y = Y[:, 0] # Select one station

            #---
            # Handle NaN Values
            #---

            # Insert numerical value that is not in data.
            # ML will hopefully recognize it.
            X[np.where(np.isnan(X))] = -999

            print("Data is prepared as follows")
            print(f"X.shape : {X.shape}")
            print(f"y.shape : {y.shape}")

            return X, y

#---
def separate_predictors(importance, n_pred_features):
    """
    Description: 
        Separates the importance of a single predictor from all features when multiple predictors were passed to the model
    Parameters:
        importance (np.array): Importance values of all features passed to model, Shape:(n_features,)
        n_pred_features (int): number of features of a single predictor within all features
    Returns:
        predictor_importance (np.array): Importance of a single predictor from a model run, Shape:(n_predictors, n_pred_features)
    Note: 
        From rf009 combinations of predictors and timelags are allowed. This function orders the n_pred_features as follows: 
        1: timelag1,pred1, 2:timelag1,pred2, ..., n: timelag1,pred_n, n+1: timelag2, pred1, ... t*n: timelag_t, pred_n
        For an example see notebooks>rf009.ipynb
    """

    # Separate importance of each predictor
    predictor_importance = []
    n_features = importance.shape[1]
    n_predictors = n_features // n_pred_features

    start = 0
    for i in range(n_predictors):
        end = start + n_pred_features 
        pred_importance = importance[:, start : end]
        start = end # Update index for next predictor
        
        predictor_importance.append(pred_importance)

    predictor_importance = np.array(predictor_importance, dtype=object) # If number of lon lats is not the same (can this happen?)

    return predictor_importance

In [ ]:
from models import loader
#---
# Initialize
#---
subregion = "lon-0530_lat7040" 
season = "winter"
detrend_type = "linear"
percentile = 0.95
is_scaled = True
model_run_flag = {
    "hanko-han-fin-cmems" : "FIN", 
    "forsmark-for-swe-cmems" : "WSWE", 
    "kalixstoron-kal-swe-cmems" : "NSWE", 
    "hamina-ham-fin-cmems" : "FINBAY", 
    "oskarshamn-osk-swe-cmems" : "WSWE2",
    "daugavgriva-dau-lva-cmems" : "LVA", 
    "travemuende-tra-deu-cmems" : "DEU",
}
range_of_years = "1999-2008"
lats, lons = preprocessing.get_lonlats(
    range_of_years="1999-2008",
    subregion="lon-0530_lat7040",
    season="winter",
    predictor="sp", # Does not matter which predictor. All predictors are sampled on same lon-lat field.
    era5_import="preprocess1",
    )

n_pred_features = len(lons) * len(lats) 
nlevels = 10 # For contourplot of predictor maps

colorbar_range = { # vmin vmax values for colorbar of predictor maps
    'sp': np.array([ 980., 1020.,]),  # Low pressure systems <980hPa (see Theory Part)
    'tp': np.array([0.    , 0.0018]),
    'u10': np.array([-17.2,  17.2]), # Storm is defined by wind stronger than 17.2m/s
    'v10': np.array([-17.2,  17.2]),
    }

In [ ]:
model_run = "rf022"

predictors_of_model = [ # Note!: If prefilling "pf" is used, they have to be always stated after ERA5 predictors

    ["sp", "tp", "u10", "v10", "pf",],
    ["sp", "tp", "u10", "v10", "pf",],

]
timelags_of_model = [ # Note!: If pf is used alone as a predictor, timelags need to be in hours instead of days
    # [0, 2, 7], 
    # [7 * 24,]
    #  
    # [3, 7*24, 5*24, 2*24],

    [1, 1, 1, 1, 1*24,],
    [2, 2, 2, 2, 2*24,],
]

runids_of_model = [
    0,
    1,
]
station_names_of_model = [
    ["kalixstoron-kal-swe-cmems",], #  NSWE
    ["hanko-han-fin-cmems",], # FIN
    ["hamina-ham-fin-cmems",], # FINBAY
    ["daugavgriva-dau-lva-cmems",], # LVA
    ["travemuende-tra-deu-cmems",], #  DEU
    ["oskarshamn-osk-swe-cmems"], #  WSWE2 
    ["forsmark-for-swe-cmems",], # WSWE
]

for station_names in station_names_of_model:
    for idx, predictors in enumerate(predictors_of_model):
        if all(x in ["daugavgriva-dau-lva-cmems", "travemuende-tra-deu-cmems"] for x in station_names): # Those stations only record from 2005-2020
            range_of_years = "2009-2018"
        else:
            range_of_years = "1999-2008" # ["1999-2008", "2009-2018", "2019-2022",] 
        
        is_pf_combined = preprocessing.check_combination(predictors)
        X = []
        Y = []
        timelags = timelags_of_model[idx]
        pred_units = {
            "sp" : "hPa",
            "tp" : "m",
            "u10" : "m/s",
            "v10" : "m/s",
            "pf"  : "m"
        }
        cmap = {
            "sp" : "coolwarm",
            "tp"  : "Blues",
            "u10" : "seismic",
            "v10" : "seismic",
        }
        station_id = model_run_flag[station_names[0]]
        model_path = f"models/random_forest/{model_run}_{station_id}/{model_run}_{station_id}_RandomSearchCV_{runids_of_model[idx]}.sav"
        
        gesla_predictand, station_positions = load_GESLA(station_names, season, detrend_type, percentile)
        if is_pf_combined==True:
            ts = preprocessing.intersect_all_times(predictors, gesla_predictand, range_of_years, subregion, season, preprocess="preprocess1")
        
        era5_counter = 0
        for pred_idx, predictor in enumerate(predictors):
            #---
            # Load Datasets of predictand and predictors
            #---
            if predictor == "pf":
                is_prefilled = True

                era5_predictor = data_loader.load_pf(season)

                if is_pf_combined:
                    X_ = preprocessing.get_timeseries(era5_predictor, ts, is_prefilling=True)
                    Y_ = preprocessing.get_timeseries(gesla_predictand, ts, is_prefilling=True)
                else:
                    # If pf is used without any ERA5 data, use hourly data of Degerby.
                    X_, Y_, t_ = preprocessing.intersect_time(era5_predictor, gesla_predictand, is_prefilled) 

            else: 
                is_prefilled = False
                era5_counter = era5_counter + 1
                era5_predictor = load_ERA5(range_of_years, subregion, season, predictor, preprocess="preprocess1")
                unit = pred_units[predictor] 
                if is_pf_combined:
                    X_ = preprocessing.get_timeseries(era5_predictor, ts, is_prefilling=False)
                    Y_ = preprocessing.get_timeseries(gesla_predictand, ts, is_prefilling=True)
                else:
                    X_, Y_, t_ = preprocessing.intersect_time(era5_predictor, gesla_predictand, is_prefilled)
            #---
            # Further preprocessing
            #---

            X_timelag, Y_timelag = preprocessing.add_timelag(X_, Y_, timelags, pred_idx) 

            X.append(X_timelag)
            Y.append(Y_timelag)

        if is_pf_combined:
            Y = np.array(Y) 
            Y = Y[0, :] # Assume all timeseries are the same for the predictors.
            ndim = Y.shape[0]
            max_length = len(X)
            n_pfs = max_length - era5_counter # Number of prefilling predictors

            print(era5_counter, max_length, n_pfs, ndim)

            era5_x = np.array(X[:era5_counter])
            era5_x = era5_x.swapaxes(0, 1)
            era5_x = era5_x.reshape(ndim, -1)

            print(f"ERA5 shape: {era5_x.shape}")

            #--- 
            # Insert prefilling data at the beginning
            #---
            XX = era5_x
            for i in range(era5_counter, max_length):
                XX = np.append(XX, X[i], axis=1)


            y = Y[:, 0] # Select one station

            #---
            # Handle NaN Values
            #---

            # Insert numerical value that is not in data.
            # ML will hopefully recognize it.
            XX[np.where(np.isnan(XX))] = -999

            X = XX
            
            del XX
        else:
            n_pfs = 0
            X, y = convert_data(X, Y,)

        #---
        # Train Test split
        #---
        X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0, test_size=0.25)

        #---
        # Scale data if they are on different scales
        #---
        X_test_unscaled = X_test

        if is_scaled:
            print("Scale training data")
            s = StandardScaler()
            s.fit(X_train)
            X_train = s.transform(X_train)
            X_test = s.transform(X_test)

        #---
        # Make Prediction
        #---
        model = loader.load_model(model_path)
        y_test_pred = model.predict(X_test)
        
        #---
        # Get days of TPs, FPs, FNs
        #---
        # Filter TP (3), FN (2), FP(1), TN(0)
        y_filtered = 2 * y_test + y_test_pred # Filter 
        tp_idx = np.where(y_filtered == 3)
        fn_idx = np.where(y_filtered == 2)

        if is_pf_combined:
            X_test_unscaled_era5 = X_test_unscaled[:, :-n_pfs]
            X_test_pred = separate_predictors(X_test_unscaled_era5, n_pred_features) # Plot only importance map of era5 data
        else:
            X_test_pred = separate_predictors(X_test_unscaled, n_pred_features)
        X_test_tp = np.mean(X_test_pred[:, tp_idx[0], :], axis=1) # (4, 17061) e.g. (pred, nlat * nlon)
        X_test_fn = np.mean(X_test_pred[:, fn_idx[0], :], axis=1)

        for pred_idx, predictor in enumerate(predictors):            
            # #---
            # # Calculate mean of predictor maps
            # #---
            # X_test_tp = np.mean(X_test[tp_idx], axis=0)
            # X_test_fn = np.mean(X_test[fn_idx], axis=0)

            #---
            # Plot figure
            #---
            if predictor in ["sp", "tp", "u10", "v10",]: # Filter if only certain predictors should be plotted ["sp", "tp", "u10", "v10"]
                tlag = timelags[pred_idx]

                data1 = X_test_fn[pred_idx, :] - X_test_tp[pred_idx, :]
                data2 = X_test_tp[pred_idx, :]
                data3 = X_test_fn[pred_idx, :]

                vmin = min(data1)
                vmax = max(data1)
                tflag = f"""Difference of mean predictor maps 
                for {predictor} with timelag {tlag}"""

                fig1, ax1 = visualisation.map(data1, lons, lats, tflag=tflag, unit=unit, vmin=vmin, vmax=vmax, nlevels=nlevels, cmap=cmap[predictor], pad=None)
                
                vmin, vmax = colorbar_range[predictor]
                tflag = f"""Mean true positive predictor map
                for {predictor} with timelag {tlag}"""

                fig2, ax2 = visualisation.map(data2, lons, lats, tflag=tflag, unit=unit, vmin=vmin, vmax=vmax, nlevels=nlevels, cmap=cmap[predictor], pad=None)

                vmin, vmax = colorbar_range[predictor]
                tflag = f"""Mean false negative predictor map
                for {predictor} with timelag {tlag}"""

                fig3, ax3 = visualisation.map(data3, lons, lats, tflag=tflag, unit=unit, vmin=vmin, vmax=vmax, nlevels=nlevels, cmap=cmap[predictor], pad=None)
                
                # Add position of station to map
                #---
                for station_name in station_names:
                    visualisation.plot_station(ax1, station_positions, station_name, is_station_name=False, is_legend=False)
                    visualisation.plot_station(ax2, station_positions, station_name, is_station_name=False, is_legend=False)
                    visualisation.plot_station(ax3, station_positions, station_name, is_station_name=False, is_legend=False)

                # Add importance to map
                #---
                folder = f"results/random_forest/{model_run}_{station_id}/"
                importance = np.load(f"{folder}importance_95_{runids_of_model[idx]}.npy")
                era5_importance = importance[:-n_pfs]
                predictor_importances = evaluation.separate_predictor_importance(era5_importance, n_pred_features)
                pred_importance = predictor_importances[pred_idx]

                evaluation.overlay_importance(ax1, pred_importance, lats, lons, percentile=99, alpha=0.08, markersize=5, color="k")
                evaluation.overlay_importance(ax2, pred_importance, lats, lons, percentile=99, alpha=0.08, markersize=5, color="k")
                evaluation.overlay_importance(ax3, pred_importance, lats, lons, percentile=99, alpha=0.08, markersize=5, color="k")

                # Save figure
                #---
                folder = f"results/random_forest/{model_run}_{station_id}/"
                saver.directory_existance(folder)

                fname1 = f"meanmap_diff_{predictor}_tlag{tlag}_{runids_of_model[idx]}"
                fname2 = f"meanmap_TP_{predictor}_tlag{tlag}_{runids_of_model[idx]}"
                fname3 = f"meanmap_FN_{predictor}_tlag{tlag}_{runids_of_model[idx]}"
                
                fig1.savefig(f"{folder}{fname1}.pdf")
                print(f"Saved Figure to {folder}{fname1}")
                fig2.savefig(f"{folder}{fname2}.pdf")
                print(f"Saved Figure to {folder}{fname2}")
                fig3.savefig(f"{folder}{fname3}.pdf")
                print(f"Saved Figure to {folder}{fname3}")